In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Glioblastoma"
cohort = "GSE178236"

# Input paths
in_trait_dir = "../../input/GEO/Glioblastoma"
in_cohort_dir = "../../input/GEO/Glioblastoma/GSE178236"

# Output paths
out_data_file = "../../output/preprocess/Glioblastoma/GSE178236.csv"
out_gene_data_file = "../../output/preprocess/Glioblastoma/gene_data/GSE178236.csv"
out_clinical_data_file = "../../output/preprocess/Glioblastoma/clinical_data/GSE178236.csv"
json_path = "../../output/preprocess/Glioblastoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Genome-wide gene expression analysis in 84 neurospheres derived from as many primary glioblastomas"
!Series_summary	"Gene expression analysis of neurosphere from primary glioblastoma tissue"
!Series_overall_design	"Total RNA obtained from neurospheres. Gene expression profile has been performed to investigate biological properties in neurospheres isolated from independent patients"
Sample Characteristics Dictionary:
{0: ['cell type: neurosphere'], 1: ['gender: male', 'gender: female'], 2: ['age: 57', 'age: 66', 'age: 17', 'age: 76', 'age: 73', 'age: 63', 'age: 67', 'age: 72', 'age: 49', 'age: 54', 'age: 62', 'age: 53', 'age: 27', 'age: 52', 'age: 48', 'age: 74', 'age: 55', 'age: 50', 'age: 65', 'age: 70', 'age: 61', 'age: 71', 'age: 47', 'age: 64', 'age: 68', 'age: 60', 'age: 45', 'age: 37', 'age: 58', 'age: 75'], 3: ['neurosphere derivation time: First surgery'], 4: ['previous treatments: Treatment naive'], 5: ['idh status: wt', 'idh status: mut'

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the Series information, this dataset contains gene expression data from glioblastoma neurospheres
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait: We can infer that all samples have glioblastoma from the Series description
# For gender: It's available at key 1
# For age: It's available at key 2
trait_row = 0  # We'll use cell type: neurosphere as a proxy for trait
gender_row = 1
age_row = 2

# 2.2 Data Type Conversion Functions
def convert_trait(trait_str):
    # Since all samples are glioblastoma neurospheres
    return 1  # All samples have the trait

def convert_gender(gender_str):
    # Extract value after colon
    if gender_str is None:
        return None
    
    gender_value = gender_str.split(":")[-1].strip().lower()
    
    if "female" in gender_value:
        return 0
    elif "male" in gender_value:
        return 1
    else:
        return None

def convert_age(age_str):
    # Extract the numeric age value
    if age_str is None:
        return None
    
    try:
        # Extract number after colon
        age_value = age_str.split(":")[-1].strip()
        return float(age_value)
    except:
        return None

# 3. Save Metadata 
# All samples have glioblastoma as indicated by the series description
is_trait_available = True

# Initial filtering of dataset usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Extract available clinical features
clinical_features = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Preview the data
preview = preview_df(clinical_features)
print("Clinical Data Preview:")
print(preview)

# Save the clinical data
clinical_features.to_csv(out_clinical_data_file, index=True)
print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{'GSM5385026': [1.0, 57.0, 1.0], 'GSM5385027': [1.0, 57.0, 1.0], 'GSM5385028': [1.0, 66.0, 0.0], 'GSM5385029': [1.0, 66.0, 0.0], 'GSM5385030': [1.0, 17.0, 0.0], 'GSM5385031': [1.0, 17.0, 0.0], 'GSM5385032': [1.0, 76.0, 1.0], 'GSM5385033': [1.0, 76.0, 1.0], 'GSM5385034': [1.0, 73.0, 1.0], 'GSM5385035': [1.0, 73.0, 1.0], 'GSM5385036': [1.0, 63.0, 1.0], 'GSM5385037': [1.0, 63.0, 1.0], 'GSM5385038': [1.0, 67.0, 1.0], 'GSM5385039': [1.0, 67.0, 1.0], 'GSM5385040': [1.0, 72.0, 1.0], 'GSM5385041': [1.0, 72.0, 1.0], 'GSM5385042': [1.0, 49.0, 1.0], 'GSM5385043': [1.0, 49.0, 1.0], 'GSM5385044': [1.0, 54.0, 1.0], 'GSM5385045': [1.0, 54.0, 1.0], 'GSM5385046': [1.0, 62.0, 1.0], 'GSM5385047': [1.0, 62.0, 0.0], 'GSM5385048': [1.0, 62.0, 0.0], 'GSM5385049': [1.0, 53.0, 1.0], 'GSM5385050': [1.0, 53.0, 1.0], 'GSM5385051': [1.0, 63.0, 1.0], 'GSM5385052': [1.0, 63.0, 1.0], 'GSM5385053': [1.0, 27.0, 1.0], 'GSM5385054': [1.0, 27.0, 1.0], 'GSM5385055': [1.0, 52.0, 1.0], 'GSM5385056': [1

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 63
Header line: "ID_REF"	"GSM5385026"	"GSM5385027"	"GSM5385028"	"GSM5385029"	"GSM5385030"	"GSM5385031"	"GSM5385032"	"GSM5385033"	"GSM5385034"	"GSM5385035"	"GSM5385036"	"GSM5385037"	"GSM5385038"	"GSM5385039"	"GSM5385040"	"GSM5385041"	"GSM5385042"	"GSM5385043"	"GSM5385044"	"GSM5385045"	"GSM5385046"	"GSM5385047"	"GSM5385048"	"GSM5385049"	"GSM5385050"	"GSM5385051"	"GSM5385052"	"GSM5385053"	"GSM5385054"	"GSM5385055"	"GSM5385056"	"GSM5385057"	"GSM5385058"	"GSM5385059"	"GSM5385060"	"GSM5385061"	"GSM5385062"	"GSM5385063"	"GSM5385064"	"GSM5385065"	"GSM5385066"	"GSM5385067"	"GSM5385068"	"GSM5385069"	"GSM5385070"	"GSM5385071"	"GSM5385072"	"GSM5385073"	"GSM5385074"	"GSM5385075"	"GSM5385076"	"GSM5385077"	"GSM5385078"	"GSM5385079"	"GSM5385080"	"GSM5385081"	"GSM5385082"	"GSM5385083"	"GSM5385084"	"GSM5385085"	"GSM5385086"	"GSM5385087"	"GSM5385088"	"GSM5385089"	"GSM5385090"	"GSM5385091"	"GSM5385092"	"GSM5385093"	"GSM5385094"	"GSM5385095"	"GSM5385096"	"GSM5385097"	"GSM5385098"	

Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers in the previous output
# These identifiers (ILMN_1343291, ILMN_1343295, etc.) are Illumina probe IDs,
# not standard human gene symbols.
# The "ILMN_" prefix indicates these are Illumina BeadArray probes
# that need to be mapped to human gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE178236
Line 6: !Series_title = Genome-wide gene expression analysis in 84 neurospheres derived from as many primary glioblastomas
Line 7: !Series_geo_accession = GSE178236
Line 8: !Series_status = Public on Jul 27 2021
Line 9: !Series_submission_date = Jun 15 2021
Line 10: !Series_last_update_date = Oct 27 2021
Line 11: !Series_pubmed_id = 34320350
Line 12: !Series_summary = Gene expression analysis of neurosphere from primary glioblastoma tissue
Line 13: !Series_overall_design = Total RNA obtained from neurospheres. Gene expression profile has been performed to investigate biological properties in neurospheres isolated from independent patients
Line 14: !Series_type = Expression profiling by array
Line 


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180, 6510136, 7560739, 1450438, 1240647], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCATGTGATACGA

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in the gene annotation map to probe IDs and gene symbols
# From previous output:
# 'ID' column contains Illumina probe IDs like ILMN_1343048
# 'Symbol' column contains gene symbols like 'phage_lambda_genome', 'thrB'

# The gene expression data has row IDs in the format ILMN_XXXXXXX (Illumina probe IDs)
# which match the 'ID' column in the gene annotation dataframe

prob_col = 'ID'  # Column containing probe IDs
gene_col = 'Symbol'  # Column containing gene symbols

# 2. Extract the mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col=prob_col, gene_col=gene_col)
print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("Gene mapping preview:")
print(preview_df(mapping_df))

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression dataframe shape after mapping: {gene_data.shape}")
print("Gene expression data preview:")
print(preview_df(gene_data))

# Save the gene expression data to CSV
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (44837, 2)
Gene mapping preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Gene': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB']}
Gene expression dataframe shape after mapping: (21464, 113)
Gene expression data preview:
{'GSM5385026': [478.5, 578.9, 558.5, 749.8, 254.4], 'GSM5385027': [517.7, 558.9, 612.1, 718.3, 270.9], 'GSM5385028': [401.29999999999995, 678.7, 565.0, 709.4, 543.8], 'GSM5385029': [396.5, 642.4, 610.3, 705.2, 621.0], 'GSM5385030': [515.6, 593.4, 619.8, 755.2, 883.0], 'GSM5385031': [454.6, 581.8, 615.3, 691.5, 802.5], 'GSM5385032': [570.9, 556.4, 541.0, 750.7, 183.8], 'GSM5385033': [428.2, 559.3, 673.6, 714.6, 171.9], 'GSM5385034': [483.20000000000005, 583.5, 573.9, 721.6, 162.6], 'GSM5385035': [647.5, 571.8, 639.4, 728.7, 545.8], 'GSM5385036': [439.70000000000005, 541.6, 612.5, 778.4, 208.5], 'GSM5385037': [446.20000000000005, 634.

Gene expression data saved to ../../output/preprocess/Glioblastoma/gene_data/GSE178236.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Load the gene expression data saved in step 6
gene_data = pd.read_csv(out_gene_data_file, index_col=0)
print(f"Loaded gene data shape: {gene_data.shape}")

# Normalize gene symbols using NCBI Gene database
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"Sample gene symbols after normalization: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data (overwrite the previous file with normalized data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Generate and save clinical data if it doesn't exist
if not os.path.exists(out_clinical_data_file):
    print("Clinical data file not found, generating it now")
    # Get the SOFT and matrix files again
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Get the clinical data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # Define conversion functions based on the sample characteristics from step 1
    def convert_trait(value):
        if not value or ':' not in value:
            return None
        # Extract tumorsphere ID
        ts_id = value.split(":", 1)[1].strip()
        # For simplicity in this study, return 1 since all samples are glioblastoma
        return 1

    def convert_age(value):
        if not value or ':' not in value:
            return None
        # Extract age value after colon
        age_str = value.split(":", 1)[1].strip()
        try:
            # Convert to integer (continuous value)
            return int(age_str)
        except ValueError:
            return None

    def convert_gender(value):
        if not value or ':' not in value:
            return None
        # Extract gender value after colon
        gender = value.split(":", 1)[1].strip().lower()
        # Convert to binary: 0 for female, 1 for male
        if gender == 'female':
            return 0
        elif gender == 'male':
            return 1
        else:
            return None
            
    # Define row indices based on sample characteristics from step 1
    trait_row = 3  # tumorsphere ID
    age_row = 2
    gender_row = 1
    
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save the clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    clinical_df = clinical_features
else:
    clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)

print(f"Clinical data shape: {clinical_df.shape}")
print("Clinical data preview:")
print(clinical_df.head())

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, first 5 columns):")
if linked_data.shape[1] >= 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data.head())

# 4. Handle missing values
print("\nMissing values before handling:")
print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
if 'Age' in linked_data.columns:
    print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
if 'Gender' in linked_data.columns:
    print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")

gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
if gene_cols:
    print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
    print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")

cleaned_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {cleaned_data.shape}")

# 5. Evaluate bias in trait and demographic features
is_trait_biased = False
if len(cleaned_data) > 0:
    trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
    is_trait_biased = trait_biased
else:
    print("No data remains after handling missing values.")
    is_trait_biased = True

# 6. Final validation and save
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=len(normalized_gene_data) > 0, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=cleaned_data,
    note=f"Dataset contains gene expression data for {trait} analysis."
)

# 7. Save if usable
if is_usable and len(cleaned_data) > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable or empty and was not saved")

Loaded gene data shape: (21464, 113)


Gene data shape after normalization: (20259, 113)
Sample gene symbols after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS']


Normalized gene data saved to ../../output/preprocess/Glioblastoma/gene_data/GSE178236.csv
Clinical data shape: (3, 113)
Clinical data preview:
              GSM5385026  GSM5385027  GSM5385028  GSM5385029  GSM5385030  \
Glioblastoma         1.0         1.0         1.0         1.0         1.0   
Age                 57.0        57.0        66.0        66.0        17.0   
Gender               1.0         1.0         0.0         0.0         0.0   

              GSM5385031  GSM5385032  GSM5385033  GSM5385034  GSM5385035  ...  \
Glioblastoma         1.0         1.0         1.0         1.0         1.0  ...   
Age                 17.0        76.0        76.0        73.0        73.0  ...   
Gender               0.0         1.0         1.0         1.0         1.0  ...   

              GSM5385129  GSM5385130  GSM5385131  GSM5385132  GSM5385133  \
Glioblastoma         1.0         1.0         1.0         1.0         1.0   
Age                 63.0         7.0        58.0        44.0        74.0  

Data shape after handling missing values: (113, 20262)
Quartiles for 'Glioblastoma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Glioblastoma' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 49.0
  50% (Median): 61.0
  75%: 67.0
Min: 7.0
Max: 76.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 29 occurrences. This represents 25.66% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Data was determined to be unusable or empty and was not saved
